# Animated Logger Tutorial
This logger is very similar to the logger Keras uses by default. It displays the samples completed out of the total, the eastimated time remaining, and other metrics. One issue with the default logger is that it doesn't make it easy to see the progress of each metric over multiple epochs. This could be rectified by using TensorBoard, which has a variety of logging utilities. However, for smaller experiments it can be impracticle.

This logger uses UTF-8 block elements to display small graphs directly in the terminal. They are displayed as a proportion of the maximum value. This allows the graph to scale automatically and provide easily understandable results.

## Usage
By using the `ProgbarLogger` implementation from [tensorflow.keras.callbacks](https://github.com/tensorflow/tensorflow/blob/1cf0898dd4331baf93fe77205550f2c2e6c90ee5/tensorflow/python/keras/callbacks.py#L693), `AnimatedLogger` can be used in the exact same way.

One stipulation is that the `verbose=` argument of the `Model.fit` method must be set to `0` when using the `AnimatedLogger`. This prevents the default progressbar from being displayed. This is required because a callback cannot affect other callbacks.

In [1]:
# setup code for the example....
import tensorflow as tf
from tensorflow.keras import layers
tf.version.VERSION

"""# Loading Data"""

((train_data,train_labels),(eval_data,eval_labels)) = tf.keras.datasets.mnist.load_data()

IMG_SIZE = train_data.shape[1]
NUM_CLASSES = 10;

#start with input(batch size omitted):
inputs = layers.Input(shape=(IMG_SIZE,IMG_SIZE,1))

#convolutional layers and max pooling
conv1 = layers.Conv2D(32,(3,3),activation="relu")(inputs)
maxp1 = layers.MaxPooling2D((2,2))(conv1)
conv2 = layers.Conv2D(64,(3,3),activation="relu")(maxp1)
maxp2 = layers.MaxPooling2D((2,2))(conv2)
conv3 = layers.Conv2D(64,(3,3),activation="relu")(maxp2)

#flatten 3D data for Dense part of network
flat = layers.Flatten()(conv3)

#dense part of network
dense1 = layers.Dense(64,activation="relu")(flat)

#final part with softmax b/c of categorical data
outputs = layers.Dense(NUM_CLASSES,activation="softmax")(dense1)

#instantiate model
model = tf.keras.Model(inputs=inputs,outputs=outputs)

#normalize training data and evaluation data
train_data = (train_data-train_data.mean())/train_data.std()
eval_data = (eval_data-eval_data.mean())/eval_data.std()
train_data = train_data.reshape((train_data.shape[0],IMG_SIZE,IMG_SIZE,1))
eval_data = eval_data.reshape((eval_data.shape[0],IMG_SIZE,IMG_SIZE,1))

#convert labels to one-hot
train_labels = tf.one_hot(train_labels,10)
eval_labels = tf.one_hot(eval_labels,10)

model.compile(optimizer="adam",loss="categorical_crossentropy",metrics=["accuracy"],)

### AnimatedLogger callback use case
Below is an example of how the AnimatedLogger can be used. Add `AnimatedLogger()` to the list of callbacks used.

*Note:* `verbose=0`

In [2]:
"""Importing AnimatedLogger Callback"""
from animated_logger import AnimatedLogger
"""Training the model"""
model.fit(x=train_data[:1000],y=train_labels[:1000],batch_size=64,epochs=10,validation_split=0.05,verbose=0,callbacks=[AnimatedLogger()])

Epoch 1/10
950/950 [██████████████████████████████████████████████████] 3.72 secs  | loss: 1.08003 | accuracy: 0.47684 | val_loss: 1.07400 | val_accuracy: 0.66000 |           
|loss                	|accuracy            	|val_loss            	|val_accuracy        	
|████████████████████	|████████████████████	|████████████████████	|████████████████████	
Epoch 2/10
950/950 [██████████████████████████████████████████████████] 0.65 secs  | loss: 0.55514 | accuracy: 0.81263 | val_loss: 0.43036 | val_accuracy: 0.86000 |           
|loss                	|accuracy            	|val_loss            	|val_accuracy        	
|████████████████████	|███████████▋        	|████████████████████	|███████████████▎    	
|██████▏             	|████████████████████	|████████            	|████████████████████	
Epoch 3/10
950/950 [██████████████████████████████████████████████████] 0.59 secs  | loss: 0.34290 | accuracy: 0.88211 | val_loss: 0.43124 | val_accuracy: 0.92000 |           
|loss                	|accu